In [149]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from google.colab import drive # access google drive

# Data Preprocessing
drive.mount('/content/drive')
filedir = '/content/drive/MyDrive/Colab_Notebooks/'
filename = filedir + 'Wcr_GPPdaily.csv'
GPP = pd.read_csv(filename) #read in csv file

print(GPP.head(1000)) #print first 10 lines of dataframe

GPP.rename(columns={"TA_F":"TA", "SW_IN_F": "SW", "VPD_F":"VPD", "GPP_NT_VUT_REF":"GPP" }, inplace=True) #rename columns
GPP.drop("TIMESTAMP",inplace=True, axis=1) #drop timestamp column
print(GPP.head(10)) # confirm that columns are renamed and timestamp dropped

GPP.isna().sum() # check for missing values (none)
GPP.drop(GPP.index[GPP["GPP"]==-9999] , inplace=True) #drop any rows with values equal to -9999

# Interaction Terms

GPP["SWxVPD"] = GPP["SW"] * GPP["VPD"] #create new column equal to two columns multiplied
GPP["SWxTA"] = GPP["SW"] * GPP["TA"]
GPP["VPDxTA"] = GPP["VPD"] * GPP["TA"]

# Linear Regression Model SciPy Optimize

def costfunc(param, SW, VPD, TA, SWVPD, SWTA, VPDTA, GPPob):
  b0, b1, b2, b3, b4, b5, b6 = param
  GPPpred=b0+ b1*SW + b2*VPD + b3*TA + b4*SWVPD + b5*SWTA + b6*VPDTA #linear regression equation
  diff = np.mean((GPPpred-GPPob)**2) #MSE equation
  return diff

guess = [10,10,10,10,10,10,10] #set initial params
mymin = minimize(costfunc, guess, args=(GPP["SW"], GPP["VPD"], GPP["TA"], GPP["SWxVPD"], GPP["SWxTA"], GPP["VPDxTA"], GPP["GPP"])) #call minimize on function
b0, b1, b2, b3, b4, b5, b6 = mymin.x
print(mymin)

# Linear Regression sklearn

x = GPP[["SW", "VPD", "TA","SWxVPD", "SWxTA", "VPDxTA" ]]  # Independent Variables
y = GPP["GPP"]  # Target Variable
model = LinearRegression()
model.fit(x, y) # run linear regression
print("Coefficients:", model.coef_)  # get coefficients
print("Intercept:", model.intercept_)  # get intercept

# Compare models next code block


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     TIMESTAMP    TA_F  SW_IN_F  VPD_F  GPP_NT_VUT_REF
0     19990101 -20.063   72.603  0.413       -0.517364
1     19990102 -12.814   12.358  0.147       -0.094241
2     19990103 -12.625   33.132  0.128       -0.166819
3     19990104 -18.652   93.481  0.263       -0.582301
4     19990105 -20.269   45.502  0.261       -0.568240
..         ...     ...      ...    ...             ...
995   20010922  12.015  154.721  3.462        3.405020
996   20010923   7.151   90.117  2.627        2.370650
997   20010924   4.579  179.627  3.379        2.075760
998   20010925   6.722  205.829  4.549        1.853690
999   20010926  10.014  190.559  3.442        1.623480

[1000 rows x 5 columns]
       TA      SW    VPD       GPP
0 -20.063  72.603  0.413 -0.517364
1 -12.814  12.358  0.147 -0.094241
2 -12.625  33.132  0.128 -0.166819
3 -18.652  93.481  0.263 -0.582301
4 -20.269  

In [150]:
# Compare Models

x_train, x_test, y_train, y_test = train_test_split(x, y) #split data into test and training datasets
model = LinearRegression()
model.fit(x_train, y_train) # create linear regression model in training data
y_pred = model.predict(x_test) #predict GPP values using test data
r2 = r2_score(y_test, y_pred) #compare real and predicted GPP values to get R2

y_pred_scipy=[] #create empty list
x_test = x_test.reset_index(drop=True) #reset index to be able to interate through it
y_test = y_test.reset_index(drop=True)
for i in range(0,len(x_test)):
  value= 0.2555 + 0.01715 * x_test.loc[i, "SW"] - 0.7876 * x_test.loc[i, "VPD"] - 0.01344 * x_test.loc[i, "TA"] - .002876 * x_test.loc[i, "SWxVPD"] + .001563 * x_test.loc[i, "SWxTA"] + 0.04386 * x_test.loc[i, "VPDxTA"] #linear regression formula with coefficients from minimize
  y_pred_scipy.append(value) #add GPP prediction to empty list

r2_scipy = r2_score(y_test, y_pred_scipy) #r2 for scipy method

print(f"Sklearn R2: {r2}")
print(f"Scipy R2: {r2_scipy}")

print("Sklearn Coefficients:", model.intercept_, model.coef_, "vs Scipy Minimize Coefficients:[2.555e-01  1.715e-02 -7.876e-01 -1.344e-02 -2.876e-03 1.563e-03  4.386e-02]")#compare coefficents
# Coefficients are very close to one another

Sklearn R2: 0.6860029312924152
Scipy R2: 0.6865017969096552
Sklearn Coefficients: 0.24977748988446846 [ 0.01697167 -0.76502826 -0.0177103  -0.00301114  0.00160066  0.04390909] vs Scipy Minimize Coefficients:[2.555e-01  1.715e-02 -7.876e-01 -1.344e-02 -2.876e-03 1.563e-03  4.386e-02]
